In [1]:
import datetime
from Portfolio import Portfolio
import Instrument
import Strategies
import numpy as np
import copy
np.set_printoptions(suppress=True)
X = 30;daysback=250 #training window, days
portfolio_lookback_period =100
target_return = 0.20
max_risk = None
max_weight= 0.20
start_portfolio_mv = 1e6
stop_limit = -0.10

tickers = {'2016-12-09':'HYLD,HYMB,IEF,JNK,MINT'.split(","),
           '2016-12-16':'HYLD,HYMB,IEF,JNK,BND'.split(","),
           '2016-12-23':'HYLD,IEF,JNK,BND,ITR,LQD'.split(","),
          '2016-12-30':'HYLD,HYG,JNK,BND,ITR,LQD'.split(",")}

time_periods = {1: ['2016-12-09','2016-12-16'],
                2: ['2016-12-16','2016-12-23'],
                3: ['2016-12-23','2016-12-30'],
                4: ['2016-12-30','2017-01-06']
               } 


portfolio = {}
pnl_history = {} 

class PortfolioRebalancing(object):
       
    def __init__(self,**vars):
        portfolio_tickers = vars.get("portfolio_tickers",None) # top forecast tickers for this cycle
        begin_date = vars.get("begin_date",None)
        end_date = vars.get("end_date",None) #Portfolio rebalance date, also used to calculate incremental returns 
        t0 = vars.get("t0",None)     
        returns = Portfolio().getReturns(portfolio_tickers,portfolio_lookback_period,end_date)
        weights = Portfolio().calculate_portfolio_weights(cvxtype='maximize_return',
                        returns_function=returns,
                        long_only=False,
                        exp_return=target_return,
                        selected_solver='SCS',
                        max_pos_size=max_weight) 

        if t0 == 0: #first period just portfolio weights...
            print ("initial portfolio weights: {}".format(list(np.round(weights,2))))
            total_tickers_px = [round(float(Portfolio().getHistoricalPrice(i,end_date)),2) for i in portfolio_tickers]              
            pnl_history[end_date] = {'model':'lstm',
                                               'new_mv':start_portfolio_mv,
                                               'pnl_period': 0,
                                               'pnl_period_perc': 0,
                                               'total_tickers':portfolio_tickers,
                                               "new_alloc_perc": list(np.round(weights,2)),
                                                "total_tickers_px": total_tickers_px,
                                               'passed_tickers':portfolio_tickers
                                              }
            new_s = []
            drop_s = []
            ns_px = []
            old_s = portfolio_tickers
            os_newpx = total_tickers_px
            os_oldpx = total_tickers_px
            old_w = list(np.round(weights,2))
            old_mv = start_portfolio_mv
              
        else:
            new_s = set(portfolio_tickers) - set(portfolio[begin_date].get("total_tickers"))  # new tickers, not originally present in the previous period
            drop_s = set(portfolio[begin_date].get("total_tickers")) - set(portfolio_tickers)            
            ns_px = [round(float(Portfolio().getHistoricalPrice(i,end_date)),2) for i in list(new_s)] # new tickers' cur period px
            print ("list(new_s):  {}".format(list(new_s)))
            old_s =  portfolio[begin_date].get("total_tickers") # old tickers, present in the previous period 
            print ("list(old_s):  {}".format(list(old_s)))
            os_newpx = [round(float(Portfolio().getHistoricalPrice(i,end_date)),2) for i in old_s] # old tickers' cur period px
            os_oldpx = portfolio[begin_date].get("total_tickers_px") # old tickers' previous period px
            old_w =  list(np.round(portfolio[begin_date].get("new_alloc_perc"),2)) # old folio weights - needed to calculate incremental and total P&L 
            old_mv = portfolio[begin_date].get("new_mv") # get previous period ending portfolio MV
                                    
        self.rebalance(end_date,new_s, ns_px, old_s ,old_w, old_mv, os_newpx, os_oldpx, drop_s, portfolio_tickers)
 
    def rebalance(self, date, # data of rebalance as well as P&L calculation for the previous period
                  new_s, # New Tickers = New tickers from model - total_tickers (current portfolio tickers)
                  ns_px, # current period prices of New Tickers (new_s)
                  old_s, # old folio tickers
                  old_w, # old folio weights - needed to calculate incremental and total P&L 
                  old_mv, # old folio mv
                  os_newpx, # current period prices of old folio tickers (old_s)
                  os_oldpx, # previous period prices of old folio tickers (old_s)
                  drop_s,  # tickers that will be closed from the new allocation.
                  portfolio_tickers
                 ):
        global portfolio;global pnl_history
        # PREVIOUS PERFORMANCE: calculate period P&L. Only then rebalance
        old_alloc = np.round(np.array(old_w)*old_mv,2)
        chg_perc_existing = np.round(list( (np.array(os_newpx) - np.array(os_oldpx))/np.array(os_oldpx)),2)
        new_alloc = np.round(old_alloc*(1 + np.array(chg_perc_existing)) ,2)
        pnl_period = sum(new_alloc) - old_mv 
        pnl_period_perc = (sum(new_alloc) - old_mv)/ old_mv
           
        portfolio[date] = {"old_s":old_s,
                                "old_w":old_w,
                                "old_mv":old_mv,
                                "os_oldpx":os_oldpx,
                                "os_newpx":os_newpx,
                                "old_alloc":np.round(np.array(old_w)*old_mv,2),
                                "chg_perc_existing": list(np.round(
                                    (np.array(os_newpx) - np.array(os_oldpx))/np.array(os_oldpx)
                                    ,2)
                                                         ),
                                "new_alloc": list(np.round(
                                    old_alloc*(1 + np.array(chg_perc_existing))
                                                      ,2)
                                    ),
                                "pnl_period": sum(new_alloc) - old_mv,
                                "pnl_period_perc": (sum(new_alloc) - old_mv)/ old_mv,
                                "new_s": new_s,
                                "ns_px": ns_px  
                               }

        #1 Remove tickers dropped via rebalancing schedule 
        if len(drop_s) > 0:
            for t in drop_s:
                s_pos = portfolio[date].get("old_s").index(t)
                for k in portfolio[date].keys():
                    if k in ["old_s", "os_newpx", "new_alloc","chg_perc_existing"]:
                        del portfolio[date].get(k)[s_pos]  

        #2 Remove tickers dropped if their drawdown is below the limit
        flag = np.vectorize(lambda x: "close" if x <= stop_limit else "keep") #close_losers
        if len(portfolio[date].get("chg_perc_existing")) > 0:
            portfolio[date].update({"flags_stops":list(flag(portfolio[date].get("chg_perc_existing") ))})                   
            count_flags = sum(1 for x in list(portfolio[date].get("flags_stops")) if x == "close")
            while count_flags > 0:
                flag_pos = portfolio[date].get("flags_stops").index("close")
                for k in portfolio[date].keys():
                    if k in ["old_s", "os_newpx", "new_alloc","chg_perc_existing"]:
                        del portfolio[date].get(k)[flag_pos]
                count_flags -= 1   
        else:
            portfolio[date].update({"flags_stops":[]})
              
        #3 Reduce tickers if they exceeded maximum allocation limit
        portfolio[date].update({"new_mv":sum(new_alloc)})
        portfolio[date].update({"new_alloc_perc": list(np.round(np.array(portfolio[date].get("new_alloc")) / 
                                       portfolio[date].get("new_mv"),2)) })
        flag_pos = np.vectorize(lambda x: "reduce" if x > max_weight else "keep") #limit_overachivers
        if len(portfolio[date].get("new_alloc_perc")) > 0: 
            portfolio[date].update({"flags_max_pos":
                                    list(flag_pos(np.round(portfolio[date].get("new_alloc_perc"),2)))})
            count_flags = sum(1 for x in list(portfolio[date].get("flags_max_pos")) if x == "reduce")
            while count_flags > 0:
                flags_max_pos = portfolio[date].get("flags_max_pos").index("reduce")
                for k in portfolio[date].keys():
                    if k in ["old_s", "os_newpx", "new_alloc","chg_perc_existing"]:
                        del portfolio[date].get(k)[flags_max_pos]
                count_flags -=1 
                
        else:
            portfolio[date].update({"flags_max_pos":[]})
   
        # add new tickers
        portfolio[date].update({"total_tickers":portfolio[date].get("old_s")})
        portfolio[date]["total_tickers"].extend(new_s)
        portfolio[date].update({"total_px":portfolio[date].get("os_newpx")})
        portfolio[date]["total_px"].extend(ns_px)           
        new_ticker_base_w = np.zeros(len(new_s)) + 0.2
        portfolio[date].update({"new_alloc_perc": 
                                       np.array( list(portfolio[date].get("new_alloc_perc")) + list(new_ticker_base_w) ) })
        
        # adjust weights to equal 100% in a portfolio        
        portfolio[date].update({"new_total_alloc": sum(portfolio[date].get("new_alloc_perc"))}) 
        portfolio[date].update({"diff": portfolio[date].get("new_total_alloc")  - 1}) 
        portfolio[date].update({"reduce_each": portfolio[date].get("diff") / 
                                       len(portfolio[date].get("total_tickers"))}) 
        portfolio[date].update({"new_alloc_perc": portfolio[date].get("new_alloc_perc") - 
                                       portfolio[date].get("reduce_each")  }) 
        portfolio[date].update({"new_alloc": portfolio[date].get("new_alloc_perc") *
                                       portfolio[date].get("new_mv") })              
        portfolio[date].update({"total_tickers_px": 
                                      [round(float(Portfolio().getHistoricalPrice(i,date)),2) 
                                       for i in portfolio[date].get("total_tickers")]}) 
         
        pnl_history[date] = {'model':'lstm',
                             "new_alloc_perc": 
                             copy.deepcopy(list(np.round(portfolio[date].get("new_alloc_perc"),2))),
                             'new_mv':copy.deepcopy(portfolio[date].get("new_mv")),
                             'pnl_period': copy.deepcopy(portfolio[date].get("pnl_period")),
                             'pnl_period_perc': 
                             copy.deepcopy(round(float(portfolio[date].get("pnl_period_perc")),4)),
                             'total_tickers': copy.deepcopy(portfolio[date].get("total_tickers")),
                             "total_tickers_px": copy.deepcopy(portfolio[date].get("total_tickers_px"))
                            }                                                        
        return True
         
     
    
if __name__ == '__main__':
    
    print ("started")
    performance_tbl = [] 
    for i, t in enumerate(time_periods):
        print (i, t)
        print ("\n\n")
        portfolio_tickers = tickers.get(time_periods.get(t)[0])
        PortfolioRebalancing(**{'portfolio_tickers':portfolio_tickers,
                         'begin_date':time_periods.get(t)[0],
                         'end_date':time_periods.get(t)[1],
                         't0':i,
                         })
    Portfolio().calculatePnLStats(pnl_history)
    print ("great success")

started
0 1



returns (72, 5)
w: var1
# The optimal expected return.
0.20000429809490675
# The optimal risk.
0.000695683007578  %
initial portfolio weights: [0.20000000000000001, 0.20000000000000001, 0.20000000000000001, 0.20000000000000001, 0.20000000000000001]
1 2



returns (72, 5)
w: var32
# The optimal expected return.
0.19996719964707893
# The optimal risk.
0.000826353877995  %
list(new_s):  ['BND']
list(old_s):  ['HYLD', 'HYMB', 'IEF', 'JNK', 'MINT']
2 3



returns (71, 6)
w: var63
# The optimal expected return.
0.20043172494919653
# The optimal risk.
0.000693725180768  %
list(new_s):  ['LQD', 'ITR']
list(old_s):  ['HYLD', 'HYMB', 'IEF', 'JNK', 'BND']
3 4



returns (70, 6)
w: var98
# The optimal expected return.
0.19965779756809274
# The optimal risk.
0.000837448505089  %
list(new_s):  ['HYG']
list(old_s):  ['HYLD', 'IEF', 'JNK', 'BND', 'LQD', 'ITR']
           model                                 alloc           mv      pnl  \
2016-12-16  lstm             [0.2, 0.2, 0.2, 0.2